In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imsave

from biobeam import SimLSM_Cylindrical
from biobeam.data import tiling
import gputools

from sim_util import sim_lightsheet_img

%matplotlib inline

# set and check which gpu is used
gputools.init_device(id_platform=0, id_device=1)
gputools.get_device()

In [ ]:
from PIL import Image

def save_tiff_stack(arr, fname):
    imlist = [Image.fromarray(arr[i]) for i in range(arr.shape[0])]
    imlist[0].save(fname, save_all=True,
                   append_images=imlist[1:])

In [ ]:
N = 512
phys_dim = 100
r_sphere = 40
grid_spacing = 20
ri_diff = 0.1
z_subsampling = 1

# half-sphere with higher ri
x = np.linspace(-phys_dim/2, phys_dim/2, N)
Xs = np.meshgrid(*([x]*3))
R = np.sqrt(np.sum([_X**2 for _X in Xs], axis = 0))
mask = (R<r_sphere) * (Xs[1] > 0)
dn = ri_diff * mask

# grid signal
x = np.arange(0, N)
Xs = np.meshgrid(*([x]*3))
grid = np.prod([_X%grid_spacing==0 for _X in Xs], axis = 0).astype(np.float32)

In [ ]:
# sim with left and right illum
image_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image_right, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))


save_tiff_stack(image_left, '/Users/david/Desktop/test_illum_left.tif')
save_tiff_stack(image_right, '/Users/david/Desktop/test_illum_right.tif')

In [ ]:
# sim with lightsheet focus at 1/4 x and 3/4 x
image_ls_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/4, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image_ls_right, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim*3/4, lam=500,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

save_tiff_stack(image_ls_left, '/Users/david/Desktop/test_lsfocus_left.tif')
save_tiff_stack(image_ls_right, '/Users/david/Desktop/test_lsfocus_right.tif')

In [ ]:
# sim at 400 & 600nm
image400nm, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=False, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=400,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

image600nm, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=True, na_illum=0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=600,
                                   z_plane_subset=np.arange(0, N, z_subsampling))

save_tiff_stack(image400nm, '/Users/david/Desktop/test_400nm.tif')
save_tiff_stack(image600nm, '/Users/david/Desktop/test_600nm.tif')

In [ ]:
# plot left/right illum
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
image_left_ = image_left[N//z_subsampling//2]
image_right_ = image_left[N//z_subsampling//2]
axs[0].imshow(1.5 * np.stack([image_left_/np.max(image_left_), image_right_/np.max(image_right_), np.zeros_like(image_left_)], 2))
axs[0].set_title('red: left, green: right')
axs[1].imshow(image_left_**(1/1), cmap='gray')
axs[1].set_title('left')
axs[2].imshow(image_right_**(1/1), cmap='gray')
axs[2].set_title('right')
fig.suptitle('left/right illumination', fontsize=20, y=0.68)
plt.savefig('/Users/david/Desktop/left_right.png', dpi=400)


# plot colors
image400nm_ = image400nm[N//z_subsampling//2]
image600nm_ = image600nm[N//z_subsampling//2]
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
axs[0].imshow(1.5 * np.stack([image400nm_/np.max(image400nm_), image600nm_/np.max(image600nm_), np.zeros_like(image400nm_)], 2))
axs[0].set_title('red: 400nm, green: 600nm')
axs[1].imshow(image400nm_**(1/1), cmap='gray')
axs[1].set_title('400 nm')
axs[2].imshow(image600nm_**(1/1), cmap='gray')
axs[2].set_title('600 nm')
fig.suptitle('wavelengths', fontsize=20, y=0.68)
plt.savefig('/Users/david/Desktop/lambda.png', dpi=400)


# plot colors
image_ls_left_ = image_ls_left[N//z_subsampling//2]
image_ls_right_ = image_ls_right[N//z_subsampling//2]
fig, axs = plt.subplots(ncols=3, figsize=[20,20])
axs[0].imshow(1.5 * np.stack([image_ls_left_/np.max(image_ls_left_), image_ls_right_/np.max(image_ls_right_), np.zeros_like(image_ls_left_)], 2))
axs[0].set_title('red: 1/4 x, green: 3/4 x')
axs[1].imshow(image_ls_left_**(1/1), cmap='gray')
axs[1].set_title('1/4 x')
axs[2].imshow(image_ls_right_**(1/1), cmap='gray')
axs[2].set_title('3/4 x')
fig.suptitle('left right lightsheet focus', fontsize=20, y=0.68)
plt.savefig('/Users/david/Desktop/ls_pos.png', dpi=400)


In [ ]:
# old code
# only single plane sim here

In [ ]:
# sim from left
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.5, size=(phys_dim,)*3, n0=1.33)
image_left = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]


image_left, _ = sim_lightsheet_img(img=grid, desc=None, dn=dn,
                                   right_illum=False, na_illum==0.1,
                                   na_detect=0.5, physical_dims=(phys_dim,)*3,
                                   ls_pos=phys_dim/2, lam=500)

# sim from right
m = SimLSM_Cylindrical(dn=np.flip(dn,2), signal=np.flip(grid,2), NA_illum=0.1, NA_detect=0.5, size=(phys_dim,)*3, n0=1.33)
image_right = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]
# flip back
image_right = np.flip(image_right, 1)

imsave('/Users/david/Desktop/test_illum_left.tif', image_left)
imsave('/Users/david/Desktop/test_illum_right.tif', image_right)

In [ ]:
# sim left ls pos
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       zfoc_illum=phys_dim/4)
image_ls_left = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

# sim right ls pos
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       zfoc_illum=phys_dim*3/4)
image_ls_right = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

imsave('/Users/david/Desktop/test_lsfocus_left.tif', image_ls_left)
imsave('/Users/david/Desktop/test_lsfocus_right.tif', image_ls_right)

In [ ]:
# sim middle at 400nm
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       lam_detect=0.4, lam_illum=0.4)
image400nm = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]
# sim middle at 600nm
m = SimLSM_Cylindrical(dn=dn, signal=grid, NA_illum=0.1, NA_detect=0.4, size=(phys_dim,)*3, n0=1.33,
                       lam_detect=0.6, lam_illum=0.6)
image600nm = m.simulate_image_z(cz=0, psf_grid_dim=(16,16),conv_sub_blocks=(2,2))[16]

imsave('/Users/david/Desktop/test_400nm.tif', image400nm)
imsave('/Users/david/Desktop/test_600nm.tif', image600nm)